In [5]:
filepath = "~/Desktop/Desktop/epidemiology_PhD/R/packages/walkboutr/tests/testthat/fixtures/"

In [ ]:
import numpy as np 
from numpy.random import RandomState
import pandas as pd
import random
from typing import Tuple

def next_lat_long(latitude: np.ndarray, longitude: np.ndarray, speed: np.ndarray, direction: np.ndarray, dt: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """Calculates the next latitude and longitude given the current position, speed, direction, and time step.

    Parameters
    ----------
    latitude : np.ndarray
        The current latitude in decimal degrees.
    longitude : np.ndarray
        The current longitude in decimal degrees.
    speed : np.ndarray
        The speed in kilometers per hour.
    direction : np.ndarray
        The direction in radians from due east.
    dt : np.ndarray
        The time step in seconds.

    Returns
    -------
    Tuple[np.ndarray, np.ndarray]
        A tuple containing the next latitude and longitude in decimal degrees.

    Raises
    ------
    ValueError
        If the speed or dt argument is negative.

    """
    if np.any(speed < 0) or np.any(dt < 0):
        raise ValueError("Speed and dt arguments must be non-negative.")

    # Convert the direction from radians to degrees.
    direction_degrees = np.degrees(direction)

    # Convert the speed from km/h to m/s.
    speed_mps = speed / 3.6

    # Calculate the distance traveled in meters.
    distance_m = speed_mps * dt

    # Calculate the bearing in degrees from due north.
    bearing_degrees = (90 - direction_degrees) % 360

    # Convert the current latitude and longitude to radians.
    lat1 = np.radians(latitude)
    lon1 = np.radians(longitude)

    # Calculate the next latitude and longitude in radians.
    lat2 = np.arcsin(np.sin(lat1) * np.cos(distance_m / 6371000) +
                     np.cos(lat1) * np.sin(distance_m / 6371000) * np.cos(np.radians(bearing_degrees)))
    lon2 = lon1 + np.arctan2(np.sin(np.radians(bearing_degrees)) * np.sin(distance_m / 6371000) * np.cos(lat1),
                             np.cos(distance_m / 6371000) - np.sin(lat1) * np.sin(lat2))

    # Convert the next latitude and longitude to decimal degrees.
    lat2_degrees = np.degrees(lat2)
    lon2_degrees = np.degrees(lon2)

    return lat2_degrees, lon2_degrees


def generate_human_motion(start_lat: float, start_long: float, rng: RandomState, n_epochs: int = 60, dt_seconds: float = 5.0) -> pd.DataFrame:
    """Generates a series of latitudes, longitudes, and speeds that mimic human motion.

    Parameters
    ----------
    start_lat : float
        The starting latitude in decimal degrees.
    start_long : float
        The starting longitude in decimal degrees.
    rng : random.Random
        The random number generator to use.

    Returns
    -------
    pd.DataFrame
        A table of data with columns [time, latitude, longitude, speed].

    """
    # Set the initial location and speed.
    current_lat = start_lat
    current_long = start_long
    current_speed = rng.uniform(0.5, 1.5)  # km/h

    # Generate a series of locations and speeds.
    directions = rng.uniform(0, 2 * np.pi, n_epochs)
    dts = rng.uniform(25, 35, n_epochs)

    # Vectorize the calculations using NumPy arrays.
    lats = np.zeros(n_epochs + 1)
    lons = np.zeros(n_epochs + 1)
    speeds = np.zeros(n_epochs + 1)
    lats[0], lons[0], speeds[0] = start_lat, start_long, current_speed
    for i in range(n_epochs):
        lats[i+1], lons[i+1] = next_lat_long(lats[i], lons[i], speeds[i], directions[i], dts[i])
        speeds[i+1] = rng.uniform(0.5, 1.5)

    # Create a DataFrame with columns [time, latitude, longitude, speed].
    times = np.arange(n_epochs + 1) * dt_seconds  
    data = np.array([times, lats, lons, speeds]).T
    df = pd.DataFrame(data, columns=["time", "latitude", "longitude", "speed"])

    return df

In [8]:
df = generate_human_motion(start_lat=47.6062, start_long=122.3321, rng=RandomState(1234), n_epochs=102)
df["time"] = pd.Timestamp('2012-04-07')+pd.to_timedelta(df["time"],unit="S")
df
df.to_csv(f'{filepath}gps_data.csv', index=False)

df = generate_human_motion(start_lat=47.6062, start_long=122.3321, rng=RandomState(1234), dt_seconds=30)
df["time"] = pd.Timestamp('2012-04-07')+pd.to_timedelta(df["time"],unit="S")
df
df.to_csv(f'{filepath}gps_data_30.csv', index=False)
